# Brain Tumor Detection Using a Convolutional Neural Network

**About the Brain MRI Images dataset:**<br>
The dataset contains 2 folders: yes and no which contains 253 Brain MRI Images. The folder yes contains 155 Brain MRI Images that are tumorous and the folder no contains 98 Brain MRI Images that are non-tumorous. You can find it [here](https://www.kaggle.com/navoneel/brain-mri-images-for-brain-tumor-detection).

## Import Necessary Modules

In [ ]:
import random
import numpy as np
import tensorflow as tf
import os

# Set random seeds for reproducibility
random.seed(42)          # Python's random module
np.random.seed(42)       # NumPy (used by scikit-learn)
tf.random.set_seed(42)   # TensorFlow/Keras
os.environ['PYTHONHASHSEED'] = str(42)

In [ ]:
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from medpy.filter.smoothing import anisotropic_diffusion as ani_diff
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import seaborn as sns
import cv2
import imutils
import matplotlib.pyplot as plt
import time
from os import listdir
from tqdm import tqdm
import matplotlib.image as mpimg


%matplotlib inline

## Data Preparation & Preprocessing

In order to crop the part that contains only the brain of the image, I used a cropping technique to find the extreme top, bottom, left and right points of the brain. You can read more about it here [Finding extreme points in contours with OpenCV](https://www.pyimagesearch.com/2016/04/11/finding-extreme-points-in-contours-with-opencv/).

In [ ]:
def crop_brain_contour(image, plot=False):
    
    # Convert the image to grayscale, and blur it slightly
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Threshold the image, then perform a series of erosions +
    # dilations to remove any small regions of noise
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Find contours in thresholded image, then grab the largest one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    

    # Find the extreme points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    
    # crop new image out of the original image using the four extreme points (left, right, top, bottom)
    new_image = image[extTop[1]:extBot[1], extLeft[0]:extRight[0]]            

    if plot:
        plt.figure()

        plt.subplot(1, 2, 1)
        plt.imshow(image)
        
        plt.tick_params(axis='both', which='both', 
                        top=False, bottom=False, left=False, right=False,
                        labelbottom=False, labeltop=False, labelleft=False, labelright=False)
        
        plt.title('Original Image')
            
        plt.subplot(1, 2, 2)
        plt.imshow(new_image)

        plt.tick_params(axis='both', which='both', 
                        top=False, bottom=False, left=False, right=False,
                        labelbottom=False, labeltop=False, labelleft=False, labelright=False)

        plt.title('Cropped Image')
        
        plt.show()
    
    return new_image

In order to better understand what it's doing, let's grab an image from the dataset and apply this cropping function to see the result:

In [ ]:
ex_img = cv2.imread('../../yes/Y1.jpg')
ex_new_img = crop_brain_contour(ex_img, True)

### Load up the data:

The following function takes two arguments, the first one is a list of directory paths for the folders 'yes' and 'no' that contain the image data and the second argument is the image size, and for every image in both directories and does the following: 
1. Read the image.
2. Crop the part of the image representing only the brain.
3. Resize the image (because the images in the dataset come in different sizes (meaning width, height and # of channels). So, we want all of our images to be (240, 240, 3) to feed it as an input to the neural network.
4. Apply normalization because we want pixel values to be scaled to the range 0-1.
5. Append the image to <i>X</i> and its label to <i>y</i>.<br>

After that, Shuffle <i>X</i> and <i>y</i>, because the data is ordered (meaning the arrays contains the first part belonging to one class and the second part belonging to the other class, and we don't want that).<br>
Finally, Return <i>X</i> and <i>y</i>.

In [ ]:
def load_data(dir_list,
              image_size,
              grayscale,
              contrast_enhancement,
              anisotropic_diffusion,
              smoothing,
              bilateral_filtering,
              clip_limit=3.0,
              tile_grid_size=8,
              kernel_size=7,
              d = 5,
              sigma_color = 30,
              sigma_space = 30,
              aniso_niter = 10,
              aniso_kappa = 50,
              aniso_gamma = 0.1,
              aniso_option = 1):
    """
    Read images, resize and normalize them.
    Arguments:
        dir_list: list of strings representing file directories.
    Returns:
        X: A numpy array with shape = (#_examples, image_width, image_height, #_channels)
        y: A numpy array with shape = (#_examples, 1)
    """

    # load all images in a directory
    X = []
    y = []
    image_width, image_height = image_size

    for directory in dir_list:
        for filename in listdir(directory):
            # load the image
            image = cv2.imread(directory + '\\' + filename)

            # crop the brain and ignore the unnecessary rest part of the image
            image = crop_brain_contour(image, plot=False)

            if grayscale:
                if len(image.shape) == 3:
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # resize image
            image = cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)

            # normalize values
            image = image / 255.

            if smoothing:
                    # Create averaging kernel
                    kernel = np.ones((kernel_size, kernel_size), np.float32) / (kernel_size**2)

                    # if grayscale or len(image.shape) == 2:
                    #     # For grayscale images
                    #     image = cv2.filter2D(image, -1, kernel)
                    # else:
                    # For color images, apply to each channel separately
                    for i in range(3):
                        image[:,:,i] = cv2.filter2D(image[:,:,i], -1, kernel)

            if bilateral_filtering:
                # Convert to uint8 for bilateral filter (it requires uint8)
                image_uint8 = (image * 255).astype(np.uint8)

                # if grayscale or len(image.shape) == 2:
                #     # For grayscale images
                #     filtered = cv2.bilateralFilter(image_uint8, d, sigma_color, sigma_space)
                #     image = filtered.astype(np.float32) / 255.0
                # else:
                # For color images, apply to each channel separately
                filtered = np.zeros_like(image_uint8)
                for i in range(3):
                    filtered[:,:,i] = cv2.bilateralFilter(image_uint8[:,:,i], d, sigma_color, sigma_space)
                image = filtered.astype(np.float32) / 255.0

            if anisotropic_diffusion:
                # if grayscale or len(image.shape) == 2:
                #     # For grayscale images
                #     image = anisotropic_diffusion(
                #         image,
                #         niter=aniso_niter,
                #         kappa=aniso_kappa,
                #         gamma=aniso_gamma,
                #         option=aniso_option
                #     )
                # else:
                # For color images, apply to each channel separately
                for i in range(3):
                    image[:,:,i] = ani_diff(
                        image[:,:,i],
                        niter=aniso_niter,
                        kappa=aniso_kappa,
                        gamma=aniso_gamma,
                        option=aniso_option
                    )

            if contrast_enhancement:
                clahe = cv2.createCLAHE(clipLimit=clip_limit,
                                        tileGridSize=(tile_grid_size, tile_grid_size))

                # if grayscale or len(image.shape) == 2:
                #     image_uint8 = (image * 255).astype(np.uint8)
                #     image = clahe.apply(image_uint8).astype(np.float32) / 255.0
                # else:
                image_uint8 = (image * 255).astype(np.uint8)
                for i in range(3):
                    image[:,:,i] = clahe.apply(image_uint8[:,:,i]).astype(np.float32) / 255.0

            # Reshape grayscale images to have a channel dimension
            if grayscale or len(image.shape) == 2:
                image = np.expand_dims(image, axis=-1)

            # convert image to numpy array and append it to X
            X.append(image)

            # append a value of 1 to the target array if the image
            # is in the folder named 'yes', otherwise append 0.
            if directory[-3:] == 'yes':
                y.append([1])
            else:
                y.append([0])

    X = np.array(X)
    y = np.array(y)

    # Shuffle the data
    X, y = shuffle(X, y)

    print(f'Number of examples is: {len(X)}')
    print(f'X shape is: {X.shape}')
    print(f'y shape is: {y.shape}')

    return X, y

Load up the data that we augmented earlier in the Data Augmentation notebook.<br>
**Note:** the augmented data directory contains not only the new generated images but also the original images.

In [ ]:
def add_implant_artifact(image, min_radius=50, max_radius=100):
    """
    Add a single simulated brain implant artifact to an MRI scan.

    Args:
        image: Input MRI image
        min_radius: Minimum radius of artifact (default: 15 pixels)
        max_radius: Maximum radius of artifact (default: 30 pixels)

    Returns:
        Image with simulated implant artifact
    """
    img_with_artifact = image.copy()
    height, width = image.shape[:2]

    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) if len(image.shape) == 3 else image
    _, brain_mask = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(brain_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        brain_coords = np.column_stack(np.where(brain_mask > 0))
        if len(brain_coords) == 0:
            return img_with_artifact  # No brain detected, return original image

    # Single larger artifact
    radius = random.randint(min_radius, max_radius)

    if contours:
        # Find the largest contour (brain)
        brain_contour = max(contours, key=cv2.contourArea)
        # Get the moments of the contour
        M = cv2.moments(brain_contour)

        # Calculate the center of the brain
        if M["m00"] != 0:
            center_x = int(M["m10"] / M["m00"])
            center_y = int(M["m01"] / M["m00"])
        else:
            center_x, center_y = width // 2, height // 2

        # Calculate distance from center to place artifact randomly but within brain
        max_distance = min(width, height) // 4
        distance = random.randint(0, max_distance)
        angle = random.uniform(0, 2 * np.pi)

        x = int(center_x + distance * np.cos(angle))
        y = int(center_y + distance * np.sin(angle))

        # Ensure coordinates are within image boundaries
        x = max(radius, min(width - radius, x))
        y = max(radius, min(height - radius, y))
    else:
        # Fallback: place artifact randomly
        x = random.randint(radius, width - radius)
        y = random.randint(radius, height - radius)

    # Create the artifact (black circle)
    cv2.circle(img_with_artifact, (x, y), radius, (0, 0, 0), -1)

    # Add some blur to make it look more realistic
    img_with_artifact = cv2.GaussianBlur(img_with_artifact, (5, 5), 0)

    return img_with_artifact

def augment_data_implant(input_dir, output_dir, percentage=0.01):
    """
    Process the dataset by adding a single implant artifact to a percentage of images

    Args:
        input_dir: Input directory containing MRI images
        output_dir: Output directory for augmented images
        percentage: Percentage of images to augment (default: 0.5)
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Get list of all images in input directory
    images = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Determine which images to augment
    num_to_augment = int(len(images) * percentage)
    images_to_augment = random.sample(images, num_to_augment)

    print(f"Processing {len(images)} images, augmenting {num_to_augment}")

    for img_name in tqdm(images):
        # Read the image
        img_path = os.path.join(input_dir, img_name)
        image = cv2.imread(img_path)

        if image is None:
            print(f"Failed to read image: {img_path}")
            continue

        # Convert to RGB (OpenCV reads as BGR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # If this image is selected for augmentation, add artifact
        if img_name in images_to_augment:
            # Add a single larger artifact
            augmented = add_implant_artifact(image)

            # Save the augmented image
            output_path = os.path.join(output_dir, img_name)
            augmented_bgr = cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)
            cv2.imwrite(output_path, augmented_bgr)

            # Visualize the original and augmented images
            fig, axes = plt.subplots(1, 2, figsize=(5, 5))
            axes = axes.flatten()
            axes[0].imshow(image, cmap='gray')
            axes[0].axis('off')
            axes[1].imshow(augmented, cmap='gray')
            axes[1].axis('off')
            plt.tight_layout()
            plt.show()
        else:
            # Copy original image to output directory
            output_path = os.path.join(output_dir, img_name)
            cv2.imwrite(output_path, image)

In [ ]:
IMG_WIDTH, IMG_HEIGHT = (240, 240)

original_augmented_path = '../../augmented data/'

# augmented data (yes and no) contains both the original and the new generated examples
original_augmented_yes = original_augmented_path + 'yes'
original_augmented_no = original_augmented_path + 'no'

As we see, we have 2065 images. Each images has a shape of **(240, 240, 3)=(image_width, image_height, number_of_channels)**

### Plot sample images:

In [ ]:
def plot_example_images(dataset, exp_name):
    fig, axes = plt.subplots(1, 4, figsize=(5, 5))
    axes = axes.flatten()

    indices = [0, 1, 2, 3]

    for idx, ax in enumerate(axes):
        image = dataset[indices[idx]]
        if image.shape[-1] == 1:
            image = image.squeeze()
        ax.imshow(image, cmap='gray')
        ax.axis('off')

    plt.tight_layout()
    plt.savefig(f"images/{exp_name}.png", dpi=300)
    plt.show()

### Split the data:
Split <i>X</i> and <i>y</i> into training, validation (development) and validation sets.

In [ ]:
def split_data(X, y, test_size=0.2):
       
    """
    Splits data into training, development and test sets.
    Arguments:
        X: A numpy array with shape = (#_examples, image_width, image_height, #_channels)
        y: A numpy array with shape = (#_examples, 1)
    Returns:
        X_train: A numpy array with shape = (#_train_examples, image_width, image_height, #_channels)
        y_train: A numpy array with shape = (#_train_examples, 1)
        X_val: A numpy array with shape = (#_val_examples, image_width, image_height, #_channels)
        y_val: A numpy array with shape = (#_val_examples, 1)
        X_test: A numpy array with shape = (#_test_examples, image_width, image_height, #_channels)
        y_test: A numpy array with shape = (#_test_examples, 1)
    """
    
    X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=test_size, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42)
    
    return X_train, y_train, X_val, y_val, X_test, y_test

Some helper functions:

In [ ]:
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m}:{round(s,1)}"

In [ ]:
def compute_f1_score(y_true, prob):
    # convert the vector of probabilities to a target vector
    y_pred = np.where(prob > 0.5, 1, 0)
    
    score = f1_score(y_true, y_pred)
    
    return score

In [ ]:
def evaluate(model, X_test, y_test, X_val, y_val):
    loss, acc = model.evaluate(x=X_test, y=y_test)
    y_val_prob = model.predict(X_val)
    f1score_val = compute_f1_score(y_val, y_val_prob)
    y_test_prob = model.predict(X_test)
    f1score = compute_f1_score(y_test, y_test_prob)

    print(f"Test Loss = {loss}")
    print(f"Test Accuracy = {acc}")
    print(f"Val F1 Score: {f1score_val}")
    print(f"Test F1 Score: {f1score}")

    return {
        'test_loss': loss,
        'test_accuracy': acc,
        'val_f1_score': f1score_val,
        'test_f1_score': f1score
    }

# Build the model

In [ ]:
def build_model(input_shape):
    """
    Arugments:
        input_shape: A tuple representing the shape of the input of the model. shape=(image_width, image_height, #_channels)
    Returns:
        model: A Model object.
    """
    # Define the input placeholder as a tensor with shape input_shape.
    X_input = Input(input_shape) # shape=(?, 240, 240, 3)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((2, 2))(X_input) # shape=(?, 244, 244, 3)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X) # shape=(?, 238, 238, 32)

    # MAXPOOL
    X = MaxPooling2D((4, 4), name='max_pool0')(X) # shape=(?, 59, 59, 32)

    # MAXPOOL
    X = MaxPooling2D((4, 4), name='max_pool1')(X) # shape=(?, 14, 14, 32)

    # FLATTEN X
    X = Flatten()(X) # shape=(?, 6272)
    # FULLYCONNECTED
    X = Dense(1, activation='sigmoid', name='fc')(X) # shape=(?, 1)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='BrainDetectionModel')
    
    return model

Define the image shape:

In [ ]:
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)

# Create a directory specific to each layer depth
model_dir = f"models/"
os.makedirs(model_dir, exist_ok=True)

EPOCHS = 10

# Baseline

In [ ]:
experiment_name = "baseline"
model = build_model(IMG_SHAPE)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

X, y = load_data(
    [original_augmented_yes, original_augmented_no],
    (IMG_WIDTH, IMG_HEIGHT),
    grayscale=False,
    contrast_enhancement=False,
    anisotropic_diffusion=False,
    smoothing=False,
    bilateral_filtering=False
)

plot_example_images(X, experiment_name)

X_train, y_train, X_val, y_val, X_test, y_test = split_data(X, y, test_size=0.3)

# Static file path to ensure only the best model is saved
filepath = os.path.join(model_dir, f"{experiment_name}.keras")

# Save only the best model based on validation accuracy
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')

start_time = time.time()

# Train the model without checkpointing
model.fit(x=X_train, y=y_train, batch_size=32, epochs=EPOCHS, validation_data=(X_val, y_val), shuffle=True, callbacks=[checkpoint], verbose=1)

end_time = time.time()
execution_time = (end_time - start_time)
print(f"{experiment_name} - Elapsed time: {hms_string(execution_time)}")

evaluate(model, X_test, y_test, X_val, y_val)

# Grayscale conversion

In [ ]:
experiment_name = "grayscale"
model = build_model((IMG_WIDTH, IMG_HEIGHT, 1))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

X, y = load_data(
    [original_augmented_yes, original_augmented_no],
    (IMG_WIDTH, IMG_HEIGHT),
    grayscale=True,
    contrast_enhancement=False,
    anisotropic_diffusion=False,
    smoothing=False,
    bilateral_filtering=False
)

plot_example_images(X, experiment_name)

X_train, y_train, X_val, y_val, X_test, y_test = split_data(X, y, test_size=0.3)

# Static file path to ensure only the best model is saved
filepath = os.path.join(model_dir, f"{experiment_name}.keras")

# Save only the best model based on validation accuracy
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')

start_time = time.time()

# Train the model without checkpointing
model.fit(x=X_train, y=y_train, batch_size=32, epochs=EPOCHS, validation_data=(X_val, y_val), shuffle=True, callbacks=[checkpoint], verbose=1)

end_time = time.time()
execution_time = (end_time - start_time)
print(f"{experiment_name} - Elapsed time: {hms_string(execution_time)}")

evaluate(model, X_test, y_test, X_val, y_val)

# Contrast Enhancement

In [ ]:
param_grid = {
    'clip_limit': [2.0, 3.0, 4.0],
    'tile_grid_size': [4, 8, 12]
}

best_val_accuracy = 0
best_params = None
results = []

for params in ParameterGrid(param_grid):
    clip_limit = params['clip_limit']
    tile_grid_size = params['tile_grid_size']

    experiment_name = f"contrast_enhancement_clip{clip_limit}_grid{tile_grid_size}"
    print(f"\nRunning experiment: {experiment_name}")

    model = build_model(IMG_SHAPE)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    X, y = load_data(
        [original_augmented_yes, original_augmented_no],
        (IMG_WIDTH, IMG_HEIGHT),
        grayscale=False,
        contrast_enhancement=True,
        anisotropic_diffusion=False,
        smoothing=False,
        bilateral_filtering=False,
        clip_limit=clip_limit,
        tile_grid_size=tile_grid_size
    )

    plot_example_images(X, f"{experiment_name}_{clip_limit}_grid{tile_grid_size}")

    # Split the data
    X_train, y_train, X_val, y_val, X_test, y_test = split_data(X, y, test_size=0.3)

    # Define file path for saving the model
    filepath = os.path.join(model_dir, f"{experiment_name}.keras")
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')

    # Train the model
    start_time = time.time()
    history = model.fit(
        x=X_train, y=y_train,
        batch_size=32,
        epochs=EPOCHS,
        validation_data=(X_val, y_val),
        shuffle=True,
        callbacks=[checkpoint],
        verbose=1
    )
    end_time = time.time()
    execution_time = end_time - start_time

    # Get the best validation accuracy from this run
    val_accuracy = max(history.history['val_accuracy'])
    print(f"{experiment_name} - Elapsed time: {hms_string(execution_time)} - Val Accuracy: {val_accuracy:.4f}")

    # Evaluate on test set
    test_metrics = evaluate(model, X_test, y_test, X_val, y_val)

    # Store results
    results.append({
        'experiment_name': experiment_name,
        'clip_limit': clip_limit,
        'tile_grid_size': tile_grid_size,
        'val_accuracy': val_accuracy,
        'test_metrics': test_metrics,
        'execution_time': execution_time
    })

    # Update best parameters if this run is better
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_params = params

# Prepare data for heatmap
clip_limits = param_grid['clip_limit']
tile_grid_sizes = param_grid['tile_grid_size']
val_accuracies = np.zeros((len(clip_limits), len(tile_grid_sizes)))

for result in results:
    i = clip_limits.index(result['clip_limit'])
    j = tile_grid_sizes.index(result['tile_grid_size'])
    val_accuracies[i, j] = result['val_accuracy']

# Create heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(val_accuracies, annot=True, fmt=".4f", xticklabels=tile_grid_sizes, yticklabels=clip_limits, cmap="YlGnBu")
plt.title("Validation Accuracy for Contrast Enhancement Parameters")
plt.xlabel("Tile Grid Size")
plt.ylabel("Clip Limit")
plt.tight_layout()
plt.show()
plt.savefig(f"images/{experiment_name}_grid-search.png", dpi=300)

# Print grid search results
print("\nGrid Search Results:")
for result in results:
    print(f"Experiment: {result['experiment_name']}, Val Accuracy: {result['val_accuracy']:.4f}, "
          f"Time: {hms_string(result['execution_time'])}")

print(f"\nBest Parameters: {best_params}")
print(f"Best Validation Accuracy: {best_val_accuracy:.4f}")

# Print test metrics for the best parameters
best_result = next(r for r in results if r['clip_limit'] == best_params['clip_limit'] and r['tile_grid_size'] == best_params['tile_grid_size'])
print(f"\nTest Metrics for Best Parameters ({best_result['experiment_name']}):")
for metric_name, metric_value in best_result['test_metrics'].items():
    print(f"{metric_name}: {metric_value:.4f}")

# Average Smoothing Filtering

In [ ]:
param_grid = {
    'kernel_size': [5, 7, 9]
}

best_val_accuracy = 0
best_params = None
results = []

for params in ParameterGrid(param_grid):
    kernel_size = params['kernel_size']

    experiment_name = f"smoothing_kernel{kernel_size}"
    print(f"\nRunning experiment: {experiment_name}")

    model = build_model(IMG_SHAPE)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    X, y = load_data(
        [original_augmented_yes, original_augmented_no],
        (IMG_WIDTH, IMG_HEIGHT),
        grayscale=False,
        contrast_enhancement=False,
        anisotropic_diffusion=False,
        smoothing=True,
        bilateral_filtering=False,
        kernel_size=kernel_size
    )

    plot_example_images(X, f"{experiment_name}_{kernel_size}")

    # Split the data
    X_train, y_train, X_val, y_val, X_test, y_test = split_data(X, y, test_size=0.3)

    # Define file path for saving the model
    filepath = os.path.join(model_dir, f"{experiment_name}.keras")
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')

    # Train the model
    start_time = time.time()
    history = model.fit(
        x=X_train, y=y_train,
        batch_size=32,
        epochs=EPOCHS,
        validation_data=(X_val, y_val),
        shuffle=True,
        callbacks=[checkpoint],
        verbose=1
    )
    end_time = time.time()
    execution_time = end_time - start_time

    # Get the best validation accuracy from this run
    val_accuracy = max(history.history['val_accuracy'])
    print(f"{experiment_name} - Elapsed time: {hms_string(execution_time)} - Val Accuracy: {val_accuracy:.4f}")

    # Evaluate on test set
    test_metrics = evaluate(model, X_test, y_test, X_val, y_val)

    # Store results
    results.append({
        'experiment_name': experiment_name,
        'kernel_size': kernel_size,
        'val_accuracy': val_accuracy,
        'test_metrics': test_metrics,
        'execution_time': execution_time
    })

    # Update best parameters if this run is better
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_params = params

# Prepare data for plotting
kernel_sizes = [result['kernel_size'] for result in results]
val_accuracies = [result['val_accuracy'] for result in results]

# Create line plot
plt.figure(figsize=(10, 6))
plt.plot(kernel_sizes, val_accuracies, marker='o', linestyle='-', color='b', label='Validation Accuracy')
plt.title("Validation Accuracy vs. Kernel Size for Smoothing")
plt.xlabel("Kernel Size")
plt.ylabel("Validation Accuracy")
plt.xticks(kernel_sizes)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
plt.savefig(f"images/{experiment_name}_grid-search.png", dpi=300)

# Print grid search results
print("\nGrid Search Results:")
for result in results:
    print(f"Experiment: {result['experiment_name']}, Val Accuracy: {result['val_accuracy']:.4f}, "
          f"Time: {hms_string(result['execution_time'])}")

print(f"\nBest Parameters: {best_params}")
print(f"Best Validation Accuracy: {best_val_accuracy:.4f}")

# Print test metrics for the best parameters
best_result = next(r for r in results if r['kernel_size'] == best_params['kernel_size'])
print(f"\nTest Metrics for Best Parameters ({best_result['experiment_name']}):")
for metric_name, metric_value in best_result['test_metrics'].items():
    print(f"{metric_name}: {metric_value:.4f}")

# Bilateral Filtering

In [ ]:
param_grid = {
    'd': [5, 9],
    'sigma_color': [10, 30, 50],
    'sigma_space': [10, 30, 50]
}

best_val_accuracy = 0
best_params = None
results = []

for params in ParameterGrid(param_grid):
    d = params['d']
    sigma_color = params['sigma_color']
    sigma_space = params['sigma_space']

    experiment_name = f"bilateral_d{d}_sigmac{sigma_color}_sigmas{sigma_space}"
    print(f"\nRunning experiment: {experiment_name}")

    model = build_model(IMG_SHAPE)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    X, y = load_data(
        [original_augmented_yes, original_augmented_no],
        (IMG_WIDTH, IMG_HEIGHT),
        grayscale=False,
        contrast_enhancement=False,
        anisotropic_diffusion=False,
        smoothing=False,
        bilateral_filtering=True,
        d=d,
        sigma_color=sigma_color,
        sigma_space=sigma_space
    )

    plot_example_images(X, f"{experiment_name}_d{d}_sigmac{sigma_color}_sigmas{sigma_space}")

    # Split the data
    X_train, y_train, X_val, y_val, X_test, y_test = split_data(X, y, test_size=0.3)

    # Define file path for saving the model
    filepath = os.path.join(model_dir, f"{experiment_name}.keras")
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')

    # Train the model
    start_time = time.time()
    history = model.fit(
        x=X_train, y=y_train,
        batch_size=32,
        epochs=EPOCHS,
        validation_data=(X_val, y_val),
        shuffle=True,
        callbacks=[checkpoint],
        verbose=1
    )
    end_time = time.time()
    execution_time = end_time - start_time

    # Get the best validation accuracy from this run
    val_accuracy = max(history.history['val_accuracy'])
    print(f"{experiment_name} - Elapsed time: {hms_string(execution_time)} - Val Accuracy: {val_accuracy:.4f}")

    # Evaluate on test set
    test_metrics = evaluate(model, X_test, y_test, X_val, y_val)

    # Store results
    results.append({
        'experiment_name': experiment_name,
        'd': d,
        'sigma_color': sigma_color,
        'sigma_space': sigma_space,
        'val_accuracy': val_accuracy,
        'test_metrics': test_metrics,
        'execution_time': execution_time
    })

    # Update best parameters if this run is better
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_params = params

# Find the best 'd' value and create a heatmap for sigma_color vs. sigma_space
best_d = best_params['d']
sigma_colors = param_grid['sigma_color']
sigma_spaces = param_grid['sigma_space']
val_accuracies = np.zeros((len(sigma_colors), len(sigma_spaces)))

for result in results:
    if result['d'] == best_d:
        i = sigma_colors.index(result['sigma_color'])
        j = sigma_spaces.index(result['sigma_space'])
        val_accuracies[i, j] = result['val_accuracy']

# Create heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(val_accuracies, annot=True, fmt=".4f", xticklabels=sigma_spaces, yticklabels=sigma_colors, cmap="YlGnBu")
plt.title(f"Validation Accuracy for Bilateral Filtering (d={best_d})")
plt.xlabel("Sigma Space")
plt.ylabel("Sigma Color")
plt.tight_layout()
plt.show()
plt.savefig(f"images/{experiment_name}_grid-search.png", dpi=300)

# Print grid search results
print("\nGrid Search Results:")
for result in results:
    print(f"Experiment: {result['experiment_name']}, Val Accuracy: {result['val_accuracy']:.4f}, "
          f"Time: {hms_string(result['execution_time'])}")

print(f"\nBest Parameters: {best_params}")
print(f"Best Validation Accuracy: {best_val_accuracy:.4f}")

# Print test metrics for the best parameters
best_result = next(r for r in results if r['d'] == best_params['d'] and
                   r['sigma_color'] == best_params['sigma_color'] and
                   r['sigma_space'] == best_params['sigma_space'])
print(f"\nTest Metrics for Best Parameters ({best_result['experiment_name']}):")
for metric_name, metric_value in best_result['test_metrics'].items():
    print(f"{metric_name}: {metric_value:.4f}")

# Anisotropic Diffusion Filtering

In [ ]:
param_grid = {
    'niter': [5, 10],
    'kappa': [20, 50, 100],
    'gamma': [0.05, 0.1, 0.15]
}

best_val_accuracy = 0
best_params = None
results = []

for params in ParameterGrid(param_grid):
    niter = params['niter']
    kappa = params['kappa']
    gamma = params['gamma']

    experiment_name = f"anisotropic_niter{niter}_kappa{kappa}_gamma{gamma}"
    print(f"\nRunning experiment: {experiment_name}")

    model = build_model(IMG_SHAPE)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    X, y = load_data(
        [original_augmented_yes, original_augmented_no],
        (IMG_WIDTH, IMG_HEIGHT),
        grayscale=False,
        contrast_enhancement=False,
        anisotropic_diffusion=True,
        smoothing=False,
        bilateral_filtering=False,
        aniso_niter=niter,
        aniso_kappa=kappa,
        aniso_gamma=gamma,
        aniso_option=1
    )

    plot_example_images(X, f"{experiment_name}_niter{niter}_kappa{kappa}_gamma{gamma}")

    # Split the data
    X_train, y_train, X_val, y_val, X_test, y_test = split_data(X, y, test_size=0.3)

    # Define file path for saving the model
    filepath = os.path.join(model_dir, f"{experiment_name}.keras")
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')

    # Train the model
    start_time = time.time()
    history = model.fit(
        x=X_train, y=y_train,
        batch_size=32,
        epochs=EPOCHS,
        validation_data=(X_val, y_val),
        shuffle=True,
        callbacks=[checkpoint],
        verbose=1
    )
    end_time = time.time()
    execution_time = end_time - start_time

    # Get the best validation accuracy from this run
    val_accuracy = max(history.history['val_accuracy'])
    print(f"{experiment_name} - Elapsed time: {hms_string(execution_time)} - Val Accuracy: {val_accuracy:.4f}")

    # Evaluate on test set
    test_metrics = evaluate(model, X_test, y_test, X_val, y_val)

    # Store results
    results.append({
        'experiment_name': experiment_name,
        'niter': niter,
        'kappa': kappa,
        'gamma': gamma,
        'val_accuracy': val_accuracy,
        'test_metrics': test_metrics,
        'execution_time': execution_time
    })

    # Update best parameters if this run is better
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_params = params

# Find the best 'niter' value and create a heatmap for kappa vs. gamma
best_niter = best_params['niter']
kappas = param_grid['kappa']
gammas = param_grid['gamma']
val_accuracies = np.zeros((len(kappas), len(gammas)))

for result in results:
    if result['niter'] == best_niter:
        i = kappas.index(result['kappa'])
        j = gammas.index(result['gamma'])
        val_accuracies[i, j] = result['val_accuracy']

# Create heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(val_accuracies, annot=True, fmt=".4f", xticklabels=gammas, yticklabels=kappas, cmap="YlGnBu")
plt.title(f"Validation Accuracy for Anisotropic Diffusion (niter={best_niter}, option=1)")
plt.xlabel("Gamma")
plt.ylabel("Kappa")
plt.tight_layout()
plt.show()
plt.savefig(f"images/{experiment_name}_grid-search.png", dpi=300)

# Print grid search results
print("\nGrid Search Results:")
for result in results:
    print(f"Experiment: {result['experiment_name']}, Val Accuracy: {result['val_accuracy']:.4f}, "
          f"Time: {hms_string(result['execution_time'])}")

print(f"\nBest Parameters: {best_params}")
print(f"Best Validation Accuracy: {best_val_accuracy:.4f}")

# Print test metrics for the best parameters
best_result = next(r for r in results if r['niter'] == best_params['niter'] and
                   r['kappa'] == best_params['kappa'] and
                   r['gamma'] == best_params['gamma'])
print(f"\nTest Metrics for Best Parameters ({best_result['experiment_name']}):")
for metric_name, metric_value in best_result['test_metrics'].items():
    print(f"{metric_name}: {metric_value:.4f}")

# Auditory Brain Implant Data Augmentation

In [ ]:
implant_augmented_path = 'implant_augmented_data/'

implant_augmented_yes = implant_augmented_path + 'yes'
implant_augmented_no = implant_augmented_path + 'no'

augment_data_implant(original_augmented_yes, implant_augmented_yes, percentage=0.01)
augment_data_implant(original_augmented_no, implant_augmented_no, percentage=0.01)


experiment_name = "implant"
model = build_model(IMG_SHAPE)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

X, y = load_data(
    [implant_augmented_yes, implant_augmented_no],
    (IMG_WIDTH, IMG_HEIGHT),
    grayscale=False,
    contrast_enhancement=False,
    anisotropic_diffusion=False,
    smoothing=False,
    bilateral_filtering=False
)

X_train, y_train, X_val, y_val, X_test, y_test = split_data(X, y, test_size=0.3)

# Static file path to ensure only the best model is saved
filepath = os.path.join(model_dir, f"{experiment_name}.keras")

# Save only the best model based on validation accuracy
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')

start_time = time.time()

# Train the model without checkpointing
model.fit(x=X_train, y=y_train, batch_size=32, epochs=EPOCHS, validation_data=(X_val, y_val), shuffle=True, callbacks=[checkpoint], verbose=1)

end_time = time.time()
execution_time = (end_time - start_time)
print(f"{experiment_name} - Elapsed time: {hms_string(execution_time)}")

evaluate(model, X_test, y_test, X_val, y_val)